In [1]:
! pip install duckdb pandas

In [10]:
import pandas as pd
import duckdb

# Create a sample DataFrame (mimicking real-world shopping data)
data = {
	"order_id": [101, 102, 103, 104, 105, 106, 107, 108],
	"customer_name": ["Ethan", "Harper", "Lucas", "Mia", "James", "Amelia", "Henry", "Charlotte"],
	"product": ["Smartphone", "Smartwatch", "Blender", "TV", "Microwave", "Laptop", "Camera", "Headphones"],
	"category": ["Electronics", "Electronics", "Home", "Electronics", "Home", "Electronics", "Electronics", "Accessories"],
	"price": [899, 250, 120, 1500, 200, 1300, 700, 180],
	"quantity": [2, 4, 3, 2, 2, 1, 3, 4],
	"order_date": pd.to_datetime(["2024-03-05 11:10", "2024-03-06 13:50", "2024-03-06 07:30",
                               	"2024-03-07 19:15", "2024-03-08 10:45", "2024-03-09 14:25",
                               	"2024-03-10 09:40", "2024-03-11 16:10"]),
	"payment_method": ["Credit Card", "PayPal", "Debit Card", "Credit Card", "PayPal", "Debit Card", "Credit Card", "PayPal"]
}

df = pd.DataFrame(data)

# Display first few rows
df.head()


,order_id,customer_name,product,category,price,quantity,order_date,payment_method
0,101,Ethan,Smartphone,Electronics,899,2,2024-03-05 11:10:00,Credit Card
1,102,Harper,Smartwatch,Electronics,250,4,2024-03-06 13:50:00,PayPal
2,103,Lucas,Blender,Home,120,3,2024-03-06 07:30:00,Debit Card
3,104,Mia,TV,Electronics,1500,2,2024-03-07 19:15:00,Credit Card
4,105,James,Microwave,Home,200,2,2024-03-08 10:45:00,PayPal


In [3]:
duckdb.query("SELECT * FROM df").df()

,order_id,customer_name,product,category,price,quantity,order_date,payment_method
0,101,Ethan,Smartphone,Electronics,899,2,2024-03-05 11:10:00,Credit Card
1,102,Harper,Smartwatch,Electronics,250,4,2024-03-06 13:50:00,PayPal
2,103,Lucas,Blender,Home,120,3,2024-03-06 07:30:00,Debit Card
3,104,Mia,TV,Electronics,1500,2,2024-03-07 19:15:00,Credit Card
4,105,James,Microwave,Home,200,2,2024-03-08 10:45:00,PayPal
5,106,Amelia,Laptop,Electronics,1300,1,2024-03-09 14:25:00,Debit Card
6,107,Henry,Camera,Electronics,700,3,2024-03-10 09:40:00,Credit Card
7,108,Charlotte,Headphones,Accessories,180,4,2024-03-11 16:10:00,PayPal


In [4]:
duckdb.query("""
    SELECT SUM(price * quantity) AS total_revenue
    FROM df
""").df()


,total_revenue
0,10678.0


In [5]:
duckdb.query("""
    SELECT product, SUM(quantity) AS total_quantity
    FROM df
    GROUP BY product
    ORDER BY total_quantity DESC
    LIMIT 5
""").df()


,product,total_quantity
0,Smartwatch,4.0
1,Headphones,4.0
2,Blender,3.0
3,Camera,3.0
4,Smartphone,2.0


In [6]:
duckdb.query("""
    SELECT customer_name, SUM(price * quantity) AS total_spent
    FROM df
    GROUP BY customer_name
    ORDER BY total_spent DESC
    LIMIT 5
""").df()


,customer_name,total_spent
0,Mia,3000.0
1,Henry,2100.0
2,Ethan,1798.0
3,Amelia,1300.0
4,Harper,1000.0


In [7]:
duckdb.query("""
    SELECT payment_method, COUNT(*) AS order_count
    FROM df
    GROUP BY payment_method
    ORDER BY order_count DESC
""").df()


,payment_method,order_count
0,PayPal,3
1,Credit Card,3
2,Debit Card,2


In [8]:
duckdb.query("""
    SELECT strftime('%H', order_date) AS order_hour, COUNT(*) AS order_count
    FROM df
    GROUP BY order_hour
    ORDER BY order_hour
""").df()


,order_hour,order_count
0,07,1
1,09,1
2,10,1
3,11,1
4,13,1
5,14,1
6,16,1
7,19,1


In [9]:
duckdb.query("""
    SELECT order_date, SUM(price * quantity)
           OVER (ORDER BY order_date) AS running_revenue
    FROM df
""").df()


,order_date,running_revenue
0,2024-03-05 11:10:00,1798.0
1,2024-03-06 07:30:00,2158.0
2,2024-03-06 13:50:00,3158.0
3,2024-03-07 19:15:00,6158.0
4,2024-03-08 10:45:00,6558.0
5,2024-03-09 14:25:00,7858.0
6,2024-03-10 09:40:00,9958.0
7,2024-03-11 16:10:00,10678.0
